In [2]:
pip install opencv-python moviepy numpy

In [3]:
import moviepy.editor as mp
from moviepy.video.fx.fadein import fadein
from moviepy.video.fx.fadeout import fadeout
import cv2

In [9]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, concatenate_videoclips, CompositeVideoClip, AudioFileClip
from moviepy.video.fx.all import fadein, fadeout
from moviepy.audio.fx.all import audio_loop

# Define the interesting clips (start_time, end_time in seconds)
clips_times = [(5, 10), (30, 35), (95, 100), (105, 110), (205, 210), (210, 215)]

# Load the video
input_video_path = '/content/New Robot Makes Soldiers Obsolete (Corridor Digital).mp4'
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Function to crop to 9:16 aspect ratio
def crop_to_9_16(frame):
    aspect_ratio = 9 / 16
    new_width = int(frame_height * aspect_ratio)
    start_x = (frame_width - new_width) // 2
    cropped_frame = frame[:, start_x:start_x + new_width]
    return cropped_frame

# Extract and save clips
def save_clips(clips_times, input_video_path, output_path_prefix='clip_'):
    cap = cv2.VideoCapture(input_video_path)
    for idx, (start, end) in enumerate(clips_times):
        cap.set(cv2.CAP_PROP_POS_MSEC, start * 1000)
        out = cv2.VideoWriter(f'{output_path_prefix}{idx}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(frame_height * 9 / 16), frame_height))
        while cap.get(cv2.CAP_PROP_POS_MSEC) < end * 1000:
            ret, frame = cap.read()
            if not ret:
                break
            cropped_frame = crop_to_9_16(frame)
            out.write(cropped_frame)
        out.release()
    cap.release()

save_clips(clips_times, input_video_path)

# Load clips with moviepy
clip_paths = [f'clip_{i}.mp4' for i in range(len(clips_times))]
clips = [VideoFileClip(path) for path in clip_paths]

# Apply transitions and concatenate clips
fade_duration = 1
trans_clips = []

for i in range(len(clips) - 1):
    clip = clips[i]
    #next_clip = clips[i + 1]
    fadeout_clip = fadeout(clip, fade_duration)
    #fadein_next_clip = fadein(next_clip, fade_duration)
    trans_clips.append(fadeout_clip)
    #trans_clips.append(fadein_next_clip.set_start(clip.duration - fade_duration))

# Add the last clip
trans_clips.append(clips[-1])

# Concatenate clips with transitions
final_clip = concatenate_videoclips(trans_clips, method="compose")

# Add background music and loop it to match the video duration
background_music = AudioFileClip("/content/latin-meeting-background-music-for-video-vlog-hip-hop-version-34-sec-207016.mp3")

# Loop the audio to match the video duration
video_duration = final_clip.duration
background_music = audio_loop(background_music, duration=video_duration)

# Set the audio to the final clip
final_clip = final_clip.set_audio(background_music.set_duration(video_duration))

# Save the final video
final_clip.write_videofile("output_video.mp4", fps=fps)


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
